# replit claude sonnet related code, sift cnn descriptors

In [1]:
import os
import time
import requests
import numpy as np
import cv2
from scipy.io import loadmat
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Input, Concatenate
from sklearn.model_selection import train_test_split

def timing_decorator(func):
    def wrapper(*args, **kwargs):
        start_time = time.time()
        print(f"\nStarting {func.__name__}...")
        result = func(*args, **kwargs)
        end_time = time.time()
        duration = end_time - start_time
        print(f"Completed {func.__name__} in {duration:.2f} seconds")
        return result
    return wrapper

@timing_decorator
def load_data():
    # Load image paths
    response = requests.get('https://raw.githubusercontent.com/RyanS974/RyanS974/main/datasets/hep2cell/cells2.txt')
    image_paths = response.text.strip().split('\n')[:5000]

    # Load labels
    response = requests.get('https://raw.githubusercontent.com/RyanS974/RyanS974/main/datasets/hep2cell/labels.mat')
    with open('labels.mat', 'wb') as f:
        f.write(response.content)
    labels = loadmat('labels.mat')['labels'][:5000]

    # Load and preprocess images
    images = []
    base_url = 'https://raw.githubusercontent.com/RyanS974/RyanS974/main/datasets/hep2cell/cells/'

    for path in image_paths:
        response = requests.get(base_url + path)
        img_array = np.asarray(bytearray(response.content), dtype=np.uint8)
        img = cv2.imdecode(img_array, cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(img, (96, 96))
        img = img / 255.0
        images.append(img)

    return np.array(images), np.array(labels)

@timing_decorator
def extract_sift_descriptors(images):
    sift = cv2.SIFT_create()
    all_descriptors = []

    for img in images:
        img_uint8 = (img * 255).astype(np.uint8)
        keypoints, descriptors = sift.detectAndCompute(img_uint8, None)
        if descriptors is None:
            descriptors = np.zeros((1, 128))
        # Pad or truncate descriptors to fixed size
        if descriptors.shape[0] > 32:
            descriptors = descriptors[:32]
        else:
            pad_size = 32 - descriptors.shape[0]
            descriptors = np.pad(descriptors, ((0, pad_size), (0, 0)))
        all_descriptors.append(descriptors)

    return np.array(all_descriptors)

def create_model(input_shape, descriptor_shape):
    # Image input branch
    img_input = Input(shape=input_shape)
    x1 = Conv2D(32, (3, 3), activation='relu')(img_input)
    x1 = MaxPooling2D()(x1)
    x1 = Conv2D(64, (3, 3), activation='relu')(x1)
    x1 = MaxPooling2D()(x1)
    x1 = Flatten()(x1)

    # SIFT descriptor branch
    desc_input = Input(shape=descriptor_shape)
    x2 = Flatten()(desc_input)
    x2 = Dense(128, activation='relu')(x2)

    # Concatenate both branches
    concat = Concatenate()([x1, x2])
    x = Dense(128, activation='relu')(concat)
    output = Dense(6, activation='softmax')(x)

    model = Model(inputs=[img_input, desc_input], outputs=output)
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

@timing_decorator
def visualize_samples(images, images_with_sift):
    fig, axes = plt.subplots(2, 5, figsize=(15, 6))

    for i in range(5):
        axes[0, i].imshow(images[i], cmap='gray')
        axes[0, i].set_title(f'Original {i+1}')
        axes[0, i].axis('off')

        axes[1, i].imshow(images_with_sift[i], cmap='gray')
        axes[1, i].set_title(f'With SIFT {i+1}')
        axes[1, i].axis('off')

    plt.tight_layout()
    plt.show()

def main():
    total_start = time.time()

    # Load and preprocess data
    images, labels = load_data()

    # Extract SIFT descriptors
    descriptors = extract_sift_descriptors(images)

    # Prepare images with SIFT visualization
    sift = cv2.SIFT_create()
    images_with_sift = []
    for i in range(5):
        img = (images[i] * 255).astype(np.uint8)
        keypoints, _ = sift.detectAndCompute(img, None)
        img_with_kp = cv2.drawKeypoints(img, keypoints, None)
        images_with_sift.append(img_with_kp)

    # Visualize samples
    visualize_samples(images[:5], images_with_sift)

    # Split dataset
    X_train, X_temp, y_train, y_temp = train_test_split(images, labels, test_size=0.3, random_state=42)
    X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

    desc_train, desc_temp = train_test_split(descriptors, test_size=0.3, random_state=42)
    desc_val, desc_test = train_test_split(desc_temp, test_size=0.5, random_state=42)

    # Create and train model
    model = create_model((96, 96, 1), (32, 128))

    print("\nStarting training...")
    train_start = time.time()
    history = model.fit(
        [X_train.reshape(-1, 96, 96, 1), desc_train],
        y_train,
        validation_data=([X_val.reshape(-1, 96, 96, 1), desc_val], y_val),
        epochs=10,
        batch_size=32
    )
    print(f"Training completed in {time.time() - train_start:.2f} seconds")

    # Evaluate model
    print("\nEvaluating model...")
    test_start = time.time()
    test_loss, test_acc = model.evaluate([X_test.reshape(-1, 96, 96, 1), desc_test], y_test)
    print(f"Testing completed in {time.time() - test_start:.2f} seconds")
    print(f"Test accuracy: {test_acc:.4f}")

    total_duration = time.time() - total_start
    print(f"\nTotal execution time: {total_duration:.2f} seconds")

if __name__ == "__main__":
    main()


Starting load_data...


error: OpenCV(4.10.0) /Users/xperience/GHA-Actions-OpenCV/_work/opencv-python/opencv-python/opencv/modules/imgproc/src/resize.cpp:4152: error: (-215:Assertion failed) !ssize.empty() in function 'resize'
